In [ ]:

!wget http://nlp.stanford.edu/data/glove.6B.zip

--2024-07-11 14:18:10--  http://nlp.stanford.edu/data/glove.6B.zip
Resolving nlp.stanford.edu (nlp.stanford.edu)... 171.64.67.140
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://nlp.stanford.edu/data/glove.6B.zip [following]
--2024-07-11 14:18:10--  https://nlp.stanford.edu/data/glove.6B.zip
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://downloads.cs.stanford.edu/nlp/data/glove.6B.zip [following]
--2024-07-11 14:18:10--  https://downloads.cs.stanford.edu/nlp/data/glove.6B.zip
Resolving downloads.cs.stanford.edu (downloads.cs.stanford.edu)... 171.64.64.22
Connecting to downloads.cs.stanford.edu (downloads.cs.stanford.edu)|171.64.64.22|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 862182613 (822M) [application/zip]
Saving to: ‘glove.6B.zip’

glov

In [ ]:

!unzip glove.6B.zip

Archive:  glove.6B.zip
  inflating: glove.6B.50d.txt        
  inflating: glove.6B.100d.txt       
  inflating: glove.6B.200d.txt       
  inflating: glove.6B.300d.txt       


In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Conv1D, GlobalMaxPooling1D, Embedding, Input
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# Load the dataset
train_df = pd.read_csv('Train_data.csv')
test_df = pd.read_csv('Test_data.csv')

# Split into features and labels
X_train = train_df['clean_text']
y_train = train_df['IsHatespeech']
X_test = test_df['clean_text']
y_test = test_df['IsHatespeech']

# Tokenize the text
max_words = 2500
max_len = 128

tokenizer = Tokenizer(num_words=max_words)
tokenizer.fit_on_texts(X_train)

X_train_seq = tokenizer.texts_to_sequences(X_train)
X_test_seq = tokenizer.texts_to_sequences(X_test)

X_train_pad = pad_sequences(X_train_seq, maxlen=max_len)
X_test_pad = pad_sequences(X_test_seq, maxlen=max_len)

# Load GloVe embeddings
embedding_dim = 100
embedding_index = {}

glove_file = 'glove.6B.100d.txt'

with open(glove_file, encoding="utf8") as f:
    for line in f:
        values = line.split()
        word = values[0]
        coefs = np.asarray(values[1:], dtype='float32')
        embedding_index[word] = coefs

# Prepare embedding matrix
word_index = tokenizer.word_index
embedding_matrix = np.zeros((max_words, embedding_dim))

for word, i in word_index.items():
    if i < max_words:
        embedding_vector = embedding_index.get(word)
        if embedding_vector is not None:
            embedding_matrix[i] = embedding_vector

# Define the CNN model with GloVe embeddings
model = Sequential()
model.add(Embedding(max_words, embedding_dim, input_length=max_len))
model.add(Conv1D(128, 5, activation='relu'))
model.add(GlobalMaxPooling1D())
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))

# Load pre-trained GloVe embeddings into the Embedding layer
model.layers[0].set_weights([embedding_matrix])
model.layers[0].trainable = False

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(X_train_pad, y_train, epochs=10, batch_size=32, validation_data=(X_test_pad, y_test))

# Predict on test data
y_pred_prob = model.predict(X_test_pad)
y_pred = (y_pred_prob > 0.5).astype(int)  # Convert probabilities to binary predictions

# Calculate metrics
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)

print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1-score:", f1)


Epoch 1/10
233/233 [==============================] - 16s 61ms/step - loss: 0.5826 - accuracy: 0.6900 - val_loss: 0.5530 - val_accuracy: 0.7186
Epoch 2/10
233/233 [==============================] - 14s 59ms/step - loss: 0.4843 - accuracy: 0.7605 - val_loss: 0.5018 - val_accuracy: 0.7599
Epoch 3/10
233/233 [==============================] - 8s 32ms/step - loss: 0.4160 - accuracy: 0.8067 - val_loss: 0.5156 - val_accuracy: 0.7556
Epoch 4/10
233/233 [==============================] - 15s 63ms/step - loss: 0.3447 - accuracy: 0.8428 - val_loss: 0.5505 - val_accuracy: 0.7438
Epoch 5/10
233/233 [==============================] - 12s 50ms/step - loss: 0.2617 - accuracy: 0.8881 - val_loss: 0.5931 - val_accuracy: 0.7390
Epoch 6/10
233/233 [==============================] - 8s 36ms/step - loss: 0.2008 - accuracy: 0.9163 - val_loss: 0.6965 - val_accuracy: 0.7379
Epoch 7/10
233/233 [==============================] - 14s 60ms/step - loss: 0.1496 - accuracy: 0.9409 - val_loss: 0.8119 - val_accuracy: 0

In [ ]:

import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Conv1D, GlobalMaxPooling1D, Embedding, Input, Bidirectional, LSTM
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# Load the dataset
train_df = pd.read_csv('Train_data.csv')
test_df = pd.read_csv('Test_data.csv')

# Split into features and labels
X_train = train_df['clean_text']
y_train = train_df['IsHatespeech']
X_test = test_df['clean_text']
y_test = test_df['IsHatespeech']

# Tokenize the text
max_words = 10000  # Increase the max words to capture more vocabulary
max_len = 128

tokenizer = Tokenizer(num_words=max_words)
tokenizer.fit_on_texts(X_train)

X_train_seq = tokenizer.texts_to_sequences(X_train)
X_test_seq = tokenizer.texts_to_sequences(X_test)

X_train_pad = pad_sequences(X_train_seq, maxlen=max_len)
X_test_pad = pad_sequences(X_test_seq, maxlen=max_len)

# Load GloVe embeddings
embedding_dim = 100
embedding_index = {}

glove_file = 'glove.6B.100d.txt'

with open(glove_file, encoding="utf8") as f:
    for line in f:
        values = line.split()
        word = values[0]
        coefs = np.asarray(values[1:], dtype='float32')
        embedding_index[word] = coefs

# Prepare embedding matrix
word_index = tokenizer.word_index
embedding_matrix = np.zeros((max_words, embedding_dim))

for word, i in word_index.items():
    if i < max_words:
        embedding_vector = embedding_index.get(word)
        if embedding_vector is not None:
            embedding_matrix[i] = embedding_vector

# Define the CNN model with GloVe embeddings
model = Sequential()
model.add(Embedding(max_words, embedding_dim, input_length=max_len, weights=[embedding_matrix], trainable=True))
model.add(Conv1D(128, 5, activation='relu'))
model.add(Conv1D(128, 5, activation='relu'))
model.add(GlobalMaxPooling1D())
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(X_train_pad, y_train, epochs=10, batch_size=32, validation_data=(X_test_pad, y_test))

# Predict on test data
y_pred_prob = model.predict(X_test_pad)
y_pred = (y_pred_prob > 0.5).astype(int)  # Convert probabilities to binary predictions

# Calculate metrics
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)

print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1-score:", f1)


Epoch 1/10
233/233 [==============================] - 36s 147ms/step - loss: 0.5731 - accuracy: 0.6884 - val_loss: 0.5601 - val_accuracy: 0.7127
Epoch 2/10
233/233 [==============================] - 33s 143ms/step - loss: 0.4328 - accuracy: 0.8008 - val_loss: 0.4926 - val_accuracy: 0.7648
Epoch 3/10
233/233 [==============================] - 34s 147ms/step - loss: 0.2984 - accuracy: 0.8772 - val_loss: 0.5383 - val_accuracy: 0.7621
Epoch 4/10
233/233 [==============================] - 40s 173ms/step - loss: 0.1699 - accuracy: 0.9357 - val_loss: 0.7064 - val_accuracy: 0.7465
Epoch 5/10
233/233 [==============================] - 37s 159ms/step - loss: 0.0919 - accuracy: 0.9688 - val_loss: 0.8787 - val_accuracy: 0.7535
Epoch 6/10
233/233 [==============================] - 38s 164ms/step - loss: 0.0551 - accuracy: 0.9803 - val_loss: 1.1541 - val_accuracy: 0.7513
Epoch 7/10
233/233 [==============================] - 40s 172ms/step - loss: 0.0509 - accuracy: 0.9824 - val_loss: 1.2786 - val_ac

In [ ]:

import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Conv1D, GlobalMaxPooling1D, Embedding, Bidirectional, LSTM
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# Load the dataset
train_df = pd.read_csv('Train_data.csv')
test_df = pd.read_csv('Test_data.csv')

# Split into features and labels
X_train = train_df['clean_text']
y_train = train_df['IsHatespeech']
X_test = test_df['clean_text']
y_test = test_df['IsHatespeech']

# Tokenize the text
max_words = 10000  # Increase the max words to capture more vocabulary
max_len = 128

tokenizer = Tokenizer(num_words=max_words)
tokenizer.fit_on_texts(X_train)

X_train_seq = tokenizer.texts_to_sequences(X_train)
X_test_seq = tokenizer.texts_to_sequences(X_test)

X_train_pad = pad_sequences(X_train_seq, maxlen=max_len)
X_test_pad = pad_sequences(X_test_seq, maxlen=max_len)

# Load GloVe embeddings
embedding_dim = 100
embedding_index = {}

glove_file = 'glove.6B.100d.txt'

with open(glove_file, encoding="utf8") as f:
    for line in f:
        values = line.split()
        word = values[0]
        coefs = np.asarray(values[1:], dtype='float32')
        embedding_index[word] = coefs

# Prepare embedding matrix
word_index = tokenizer.word_index
embedding_matrix = np.zeros((max_words, embedding_dim))

for word, i in word_index.items():
    if i < max_words:
        embedding_vector = embedding_index.get(word)
        if embedding_vector is not None:
            embedding_matrix[i] = embedding_vector

# Define the improved model with CNN and Bidirectional LSTM
model = Sequential()
model.add(Embedding(max_words, embedding_dim, input_length=max_len, weights=[embedding_matrix], trainable=True))
model.add(Conv1D(128, 5, activation='relu'))
model.add(Bidirectional(LSTM(64, return_sequences=True)))
model.add(GlobalMaxPooling1D())
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))

# Compile the model
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4), loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
history = model.fit(X_train_pad, y_train, epochs=10, batch_size=32, validation_data=(X_test_pad, y_test))

# Predict on test data
y_pred_prob = model.predict(X_test_pad)
y_pred = (y_pred_prob > 0.5).astype(int)  # Convert probabilities to binary predictions

# Calculate metrics
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)

print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1-score:", f1)


Epoch 1/10
233/233 [==============================] - 85s 346ms/step - loss: 0.6235 - accuracy: 0.6410 - val_loss: 0.5666 - val_accuracy: 0.7030
Epoch 2/10
233/233 [==============================] - 71s 303ms/step - loss: 0.5366 - accuracy: 0.7288 - val_loss: 0.5460 - val_accuracy: 0.7288
Epoch 3/10
233/233 [==============================] - 76s 328ms/step - loss: 0.5050 - accuracy: 0.7565 - val_loss: 0.5248 - val_accuracy: 0.7368
Epoch 4/10
233/233 [==============================] - 77s 329ms/step - loss: 0.4762 - accuracy: 0.7737 - val_loss: 0.5330 - val_accuracy: 0.7309
Epoch 5/10
233/233 [==============================] - 69s 297ms/step - loss: 0.4393 - accuracy: 0.7981 - val_loss: 0.5071 - val_accuracy: 0.7524
Epoch 6/10
233/233 [==============================] - 72s 311ms/step - loss: 0.4113 - accuracy: 0.8153 - val_loss: 0.5145 - val_accuracy: 0.7519
Epoch 7/10
233/233 [==============================] - 76s 325ms/step - loss: 0.3803 - accuracy: 0.8345 - val_loss: 0.5225 - val_ac

In [ ]:

import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Embedding, LSTM, Bidirectional
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# Load the dataset
train_df = pd.read_csv('Train_data.csv')
test_df = pd.read_csv('Test_data.csv')

# Split into features and labels
X_train = train_df['clean_text']
y_train = train_df['IsHatespeech']
X_test = test_df['clean_text']
y_test = test_df['IsHatespeech']

# Tokenize the text
max_words = 10000  # Increase the max words to capture more vocabulary
max_len = 128

tokenizer = Tokenizer(num_words=max_words)
tokenizer.fit_on_texts(X_train)

X_train_seq = tokenizer.texts_to_sequences(X_train)
X_test_seq = tokenizer.texts_to_sequences(X_test)

X_train_pad = pad_sequences(X_train_seq, maxlen=max_len)
X_test_pad = pad_sequences(X_test_seq, maxlen=max_len)

# Load GloVe embeddings
embedding_dim = 100
embedding_index = {}

glove_file = 'glove.6B.100d.txt'

with open(glove_file, encoding="utf8") as f:
    for line in f:
        values = line.split()
        word = values[0]
        coefs = np.asarray(values[1:], dtype='float32')
        embedding_index[word] = coefs

# Prepare embedding matrix
word_index = tokenizer.word_index
embedding_matrix = np.zeros((max_words, embedding_dim))

for word, i in word_index.items():
    if i < max_words:
        embedding_vector = embedding_index.get(word)
        if embedding_vector is not None:
            embedding_matrix[i] = embedding_vector

# Define the RNN model with GloVe embeddings
model = Sequential()
model.add(Embedding(max_words, embedding_dim, input_length=max_len, weights=[embedding_matrix], trainable=True))
model.add(Bidirectional(LSTM(128, return_sequences=True)))
model.add(Bidirectional(LSTM(64)))
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))

# Compile the model
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4), loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
history = model.fit(X_train_pad, y_train, epochs=10, batch_size=32, validation_data=(X_test_pad, y_test))

# Predict on test data
y_pred_prob = model.predict(X_test_pad)
y_pred = (y_pred_prob > 0.5).astype(int)  # Convert probabilities to binary predictions

# Calculate metrics
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)

print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1-score:", f1)


Epoch 1/10
233/233 [==============================] - 280s 1s/step - loss: 0.6015 - accuracy: 0.6680 - val_loss: 0.5449 - val_accuracy: 0.7277
Epoch 2/10
233/233 [==============================] - 268s 1s/step - loss: 0.5137 - accuracy: 0.7471 - val_loss: 0.5256 - val_accuracy: 0.7331
Epoch 3/10
233/233 [==============================] - 265s 1s/step - loss: 0.4854 - accuracy: 0.7659 - val_loss: 0.5068 - val_accuracy: 0.7438
Epoch 4/10
233/233 [==============================] - 255s 1s/step - loss: 0.4589 - accuracy: 0.7859 - val_loss: 0.4976 - val_accuracy: 0.7551
Epoch 5/10
233/233 [==============================] - 260s 1s/step - loss: 0.4334 - accuracy: 0.7944 - val_loss: 0.4946 - val_accuracy: 0.7589
Epoch 6/10
233/233 [==============================] - 258s 1s/step - loss: 0.4075 - accuracy: 0.8140 - val_loss: 0.4987 - val_accuracy: 0.7583
Epoch 7/10
233/233 [==============================] - 287s 1s/step - loss: 0.3873 - accuracy: 0.8286 - val_loss: 0.5192 - val_accuracy: 0.7573

In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Embedding, GRU, Bidirectional, SpatialDropout1D
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# Load the dataset
train_df = pd.read_csv('Train_data.csv')
test_df = pd.read_csv('Test_data.csv')

# Split into features and labels
X_train = train_df['clean_text'].tolist()
y_train = train_df['IsHatespeech'].tolist()
X_test = test_df['clean_text'].tolist()
y_test = test_df['IsHatespeech'].tolist()

# Tokenize the text
max_words = 20000  # Increase the max words to capture more vocabulary
max_len = 128

tokenizer = Tokenizer(num_words=max_words)
tokenizer.fit_on_texts(X_train)

X_train_seq = tokenizer.texts_to_sequences(X_train)
X_test_seq = tokenizer.texts_to_sequences(X_test)

X_train_pad = pad_sequences(X_train_seq, maxlen=max_len)
X_test_pad = pad_sequences(X_test_seq, maxlen=max_len)

# Load GloVe embeddings
embedding_dim = 100
embedding_index = {}

glove_file = 'glove.6B.100d.txt'

with open(glove_file, encoding="utf8") as f:
    for line in f:
        values = line.split()
        word = values[0]
        coefs = np.asarray(values[1:], dtype='float32')
        embedding_index[word] = coefs

# Prepare embedding matrix
word_index = tokenizer.word_index
embedding_matrix = np.zeros((max_words, embedding_dim))

for word, i in word_index.items():
    if i < max_words:
        embedding_vector = embedding_index.get(word)
        if embedding_vector is not None:
            embedding_matrix[i] = embedding_vector

# Define the improved RNN model with GloVe embeddings
model = Sequential()
model.add(Embedding(max_words, embedding_dim, input_length=max_len, weights=[embedding_matrix], trainable=False))
model.add(SpatialDropout1D(0.2))
model.add(Bidirectional(GRU(128, return_sequences=True)))
model.add(SpatialDropout1D(0.2))
model.add(Bidirectional(GRU(64)))
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))

# Compile the model
optimizer = tf.keras.optimizers.Adam(learning_rate=2e-5)
model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])

# Learning rate scheduler
lr_scheduler = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=3, verbose=1)

# Train the model
history = model.fit(X_train_pad, np.array(y_train), epochs=10, batch_size=32,
                    validation_data=(X_test_pad, np.array(y_test)),
                    callbacks=[lr_scheduler]) # Convert y_train and y_test to NumPy arrays
# Predict on test data
y_pred_prob = model.predict(X_test_pad)
y_pred = (y_pred_prob > 0.5).astype(int)  # Convert probabilities to binary predictions

# Calculate metrics
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)

print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1-score:", f1)


Epoch 1/10
233/233 [==============================] - 178s 725ms/step - loss: 0.6846 - accuracy: 0.5473 - val_loss: 0.6704 - val_accuracy: 0.6208 - lr: 2.0000e-05
Epoch 2/10
233/233 [==============================] - 166s 715ms/step - loss: 0.6599 - accuracy: 0.6259 - val_loss: 0.6409 - val_accuracy: 0.6622 - lr: 2.0000e-05
Epoch 3/10
233/233 [==============================] - 162s 698ms/step - loss: 0.6287 - accuracy: 0.6642 - val_loss: 0.6092 - val_accuracy: 0.6901 - lr: 2.0000e-05
Epoch 4/10
233/233 [==============================] - 159s 682ms/step - loss: 0.6022 - accuracy: 0.6852 - val_loss: 0.5861 - val_accuracy: 0.6928 - lr: 2.0000e-05
Epoch 5/10
233/233 [==============================] - 171s 732ms/step - loss: 0.5802 - accuracy: 0.7015 - val_loss: 0.5743 - val_accuracy: 0.7019 - lr: 2.0000e-05
Epoch 6/10
233/233 [==============================] - 164s 704ms/step - loss: 0.5676 - accuracy: 0.7103 - val_loss: 0.5673 - val_accuracy: 0.7084 - lr: 2.0000e-05
Epoch 7/10
233/233 [==

In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Embedding, LSTM, Bidirectional, Conv1D, GlobalMaxPooling1D, SpatialDropout1D
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# Load the dataset
train_df = pd.read_csv('Train_data.csv')
test_df = pd.read_csv('Test_data.csv')

# Split into features and labels
X_train = train_df['clean_text'].tolist()
y_train = train_df['IsHatespeech'].tolist()
X_test = test_df['clean_text'].tolist()
y_test = test_df['IsHatespeech'].tolist()

# Tokenize the text
max_words = 20000  # Increase the max words to capture more vocabulary
max_len = 128

tokenizer = Tokenizer(num_words=max_words)
tokenizer.fit_on_texts(X_train)

X_train_seq = tokenizer.texts_to_sequences(X_train)
X_test_seq = tokenizer.texts_to_sequences(X_test)

X_train_pad = pad_sequences(X_train_seq, maxlen=max_len)
X_test_pad = pad_sequences(X_test_seq, maxlen=max_len)

# Load GloVe embeddings
embedding_dim = 100
embedding_index = {}

glove_file = 'glove.6B.100d.txt'

with open(glove_file, encoding="utf8") as f:
    for line in f:
        values = line.split()
        word = values[0]
        coefs = np.asarray(values[1:], dtype='float32')
        embedding_index[word] = coefs

# Prepare embedding matrix
word_index = tokenizer.word_index
embedding_matrix = np.zeros((max_words, embedding_dim))

for word, i in word_index.items():
    if i < max_words:
        embedding_vector = embedding_index.get(word)
        if embedding_vector is not None:
            embedding_matrix[i] = embedding_vector

# Define the hybrid CNN + BiLSTM model with GloVe embeddings
model = Sequential()
model.add(Embedding(max_words, embedding_dim, input_length=max_len, weights=[embedding_matrix], trainable=False))
model.add(SpatialDropout1D(0.2))
model.add(Conv1D(filters=128, kernel_size=5, activation='relu'))
model.add(Bidirectional(LSTM(128, return_sequences=True)))
model.add(Bidirectional(LSTM(64)))
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))

# Compile the model
optimizer = tf.keras.optimizers.Adam(learning_rate=2e-5)
model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])

# Learning rate scheduler
lr_scheduler = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=3, verbose=1)

# Train the model
history = model.fit(X_train_pad, np.array(y_train), epochs=10, batch_size=32,
                    validation_data=(X_test_pad, np.array(y_test)),
                    callbacks=[lr_scheduler]) # Convert y_train and y_test to NumPy arrays

# Predict on test data
y_pred_prob = model.predict(X_test_pad)
y_pred = (y_pred_prob > 0.5).astype(int)  # Convert probabilities to binary predictions

# Calculate metrics
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)

print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1-score:", f1)


Epoch 1/10
233/233 [==============================] - 291s 1s/step - loss: 0.6843 - accuracy: 0.5690 - val_loss: 0.6650 - val_accuracy: 0.6305 - lr: 2.0000e-05
Epoch 2/10
233/233 [==============================] - 273s 1s/step - loss: 0.6165 - accuracy: 0.6807 - val_loss: 0.5810 - val_accuracy: 0.6987 - lr: 2.0000e-05
Epoch 3/10
233/233 [==============================] - 278s 1s/step - loss: 0.5682 - accuracy: 0.7079 - val_loss: 0.5652 - val_accuracy: 0.7121 - lr: 2.0000e-05
Epoch 4/10
233/233 [==============================] - 273s 1s/step - loss: 0.5624 - accuracy: 0.7104 - val_loss: 0.5598 - val_accuracy: 0.7202 - lr: 2.0000e-05
Epoch 5/10
233/233 [==============================] - 295s 1s/step - loss: 0.5546 - accuracy: 0.7214 - val_loss: 0.5538 - val_accuracy: 0.7170 - lr: 2.0000e-05
Epoch 6/10
233/233 [==============================] - 278s 1s/step - loss: 0.5463 - accuracy: 0.7244 - val_loss: 0.5520 - val_accuracy: 0.7250 - lr: 2.0000e-05
Epoch 7/10
233/233 [====================